In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

C:\Users\elrii\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
input_dir = r'./input/'
output_dir = r'./output/'

In [3]:
dfa = pd.read_sas(input_dir+'AUX_I_2015.XPT')
dfq = pd.read_sas(input_dir+'AUQ_I_2015.XPT')
dfd = pd.read_sas(input_dir+'DEMO_I_2015.XPT') #CHANGE

In [4]:
#dfq.to_excel(output_dir+"hi.xlsx")

In [5]:
dfa = dfa[['SEQN', 'AUXU500R','AUXU1K2R','AUXU2KR','AUXU3KR',
            'AUXU4KR','AUXU6KR','AUXU8KR','AUXU500L',
            'AUXU1K2L','AUXU2KL','AUXU3KL','AUXU4KL','AUXU6KL',
            'AUXU8KL']].replace(666,100)

audiogram_col = ['AUXU500R','AUXU1K2R','AUXU2KR','AUXU3KR',
            'AUXU4KR','AUXU6KR','AUXU8KR','AUXU500L',
            'AUXU1K2L','AUXU2KL','AUXU3KL','AUXU4KL','AUXU6KL',
            'AUXU8KL']
audiogram_right = audiogram_col[:7]
audiogram_left = audiogram_col[7:]

dfa = dfa[~dfa[audiogram_col].isin([888]).any(axis=1)].dropna().astype(int)
dfa = dfa[(dfa[audiogram_right].sum(axis=1) >= 140) & (dfa[audiogram_right].sum(axis=1) >= 140)].reset_index(drop=True)
dfa

,SEQN,AUXU500R,AUXU1K2R,AUXU2KR,AUXU3KR,AUXU4KR,AUXU6KR,AUXU8KR,AUXU500L,AUXU1K2L,AUXU2KL,AUXU3KL,AUXU4KL,AUXU6KL,AUXU8KL
0,83732,10,5,5,20,30,25,50,5,0,5,35,50,50,55
1,83733,10,10,40,55,60,55,65,5,15,50,55,60,55,60
2,83735,25,25,30,20,40,40,55,15,25,30,30,45,60,50
3,83744,20,10,15,20,10,30,40,20,15,10,15,40,35,30
4,83747,0,10,15,20,35,40,50,10,15,25,40,40,55,60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1313,93633,20,30,25,25,45,20,40,35,40,35,50,35,35,50
1314,93640,20,20,35,40,35,55,25,15,20,40,40,30,35,20
1315,93643,35,40,50,40,40,40,65,15,20,25,20,20,55,50
1316,93685,20,25,45,50,60,65,70,15,15,35,55,90,90,105


In [6]:
dfq = dfq[['SEQN','AUQ054', 'AUQ100', 'AUQ110', 'AUQ331']] #CHECK
dfq=dfq.dropna(how='any').reset_index(drop=True)
dfq.shape

(4767, 5)

In [7]:
dfd = dfd[['SEQN', 'RIAGENDR','RIDAGEYR', 'DMQMILIZ']] #1 = male, 2 = female

thresholds = [0, 25, 35, 45, 55, dfd['RIDAGEYR'].max()] 
labels = [1, 2, 3,4,5]
dfd['RIDAGEYR'] = pd.cut(dfd['RIDAGEYR'], 
                         bins=thresholds, labels=labels, right=True)
dfd = dfd.dropna(subset=['DMQMILIZ']).reset_index(drop=True)
dfd.shape

(6149, 4)

In [8]:
dfV = pd.merge(pd.merge(dfd, dfa, on='SEQN'), dfq).astype(int)
print(dfV.shape)
dfV.to_excel(output_dir+"NHANES_2015-2016.xlsx") #CHANGE

(1318, 22)


In [9]:
dfV

,SEQN,RIAGENDR,RIDAGEYR,DMQMILIZ,AUXU500R,AUXU1K2R,AUXU2KR,AUXU3KR,AUXU4KR,AUXU6KR,...,AUXU1K2L,AUXU2KL,AUXU3KL,AUXU4KL,AUXU6KL,AUXU8KL,AUQ054,AUQ100,AUQ110,AUQ331
0,83732,1,5,2,10,5,5,20,30,25,...,0,5,35,50,50,55,1,4,5,2
1,83733,1,4,2,10,10,40,55,60,55,...,15,50,55,60,55,60,1,5,5,2
2,83735,2,5,2,25,25,30,20,40,40,...,25,30,30,45,60,50,3,3,3,2
3,83744,1,5,2,20,10,15,20,10,30,...,15,10,15,40,35,30,2,5,5,1
4,83747,1,4,2,0,10,15,20,35,40,...,15,25,40,40,55,60,1,5,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1313,93633,1,5,2,20,30,25,25,45,20,...,40,35,50,35,35,50,1,5,5,2
1314,93640,1,3,2,20,20,35,40,35,55,...,20,40,40,30,35,20,2,3,4,2
1315,93643,2,5,2,35,40,50,40,40,40,...,20,25,20,20,55,50,1,4,4,2
1316,93685,1,4,2,20,25,45,50,60,65,...,15,35,55,90,90,105,2,5,5,1
